In [ ]:
# 数据可视化
# import matplotlib.pyplot as plt
# df_data['sentence_len'].hist(bins=100)
# plt.xlim(0, 100)
# plt.xlabel('sentence_length')
# plt.ylabel('sentence_num')
# plt.title('Distribution of the Length of Sentence')
# plt.show()

In [1]:
import pandas as pd
import numpy as np
import re
from itertools import chain
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import np_utils

# filename = './data/sentiment_word_tagging_train.csv'
def word_to_index(sentence, flag, tokenizer):
    res = []
    if flag == 'content':
        tmp = tokenizer.texts_to_sequences(sentence)
        for i in tmp:
            if i:
                res.append(i[0])
    else:
        tag_dict = {'B':1, 'M':2, 'E':3, 'S':4, 'N':5}
        for word in sentence:
            res.append(tag_dict[word])
    return [res]

def get_Xy(sentence):
    """将 sentence 处理成 [word1, w2, ..wn], [tag1, t2, ...tn]"""
    sentence = sentence.replace("//", '$/')
    words_tags = re.findall('(.)/(.)', sentence)
    if words_tags:
        words_tags = np.asarray(words_tags)
        words = words_tags[:, 0]
        tags = words_tags[:, 1]
        return words, tags # 所有的字和tag分别存为 data / label
    return None

def initTokenizer():
    df = pd.read_csv('./data/allwords.csv', dtype=np.str, header=None)
    all_words = df[0].values
    tokenizer = Tokenizer(lower=False)
    tokenizer.fit_on_texts(all_words)
    return tokenizer
zy = {}
def initTranspos(transposematrix, labels):
    for label in labels:
        for i in range(1, len(label)):
            status = label[i-1] + label[i]
            if transposematrix.get(status, -1) == -1:
                transposematrix[status] = 1
            else:
                transposematrix[status] = transposematrix[status] + 1

def generate_data(filename):
    raw_data = pd.read_csv(filename, header=None, delimiter='\t')
    s = ''
    for index, row in raw_data.iterrows():
        if index != 0:
            s = s + ' '
        s = s + row.values[0]
    sentences = re.split(u'[，。！？、‘’“”]/[BMENS]', s)
    datas = []
    labels = []
    for sentence in iter(sentences):
        res = get_Xy(sentence)
        if res:
            datas.append(res[0])
            labels.append(res[1])
#     initTranspos(zy, labels)
    df_data = pd.DataFrame({'words': datas, 'tags': labels}, index=range(len(datas)))
    #　句子长度
    df_data['sentence_len'] = df_data['words'].apply(lambda words: len(words))


    tokenizer = initTokenizer()
    df_data['X'] = df_data['words'].apply(word_to_index, args = ['content', tokenizer])
    df_data['Y'] = df_data['tags'].apply(word_to_index, args = ['sentiment', tokenizer])
    print("finish word_to_index")
    
    maxlen = 40
    df_data['X'] = df_data['X'].apply(pad_sequences, args=[maxlen, 'int32', 'post'])
    df_data['Y'] = df_data['Y'].apply(pad_sequences, args=[maxlen, 'int32', 'post'])
    
    X = np.asarray(list(df_data['X'].values))
    y = np.asarray(list(df_data['Y'].values))
    X = X.reshape(X.shape[0], X.shape[2])
    y = y.reshape(y.shape[0], y.shape[2])
    
    #将标签向量one-hot
    def getY(y):
        res = []
        for row in y:
            tmp = []
            for col in row:
                tmp.append(np_utils.to_categorical(col, 6))
            res.append(tmp)
        return np.array(res)
    y = getY(y)
    y = y.reshape(-1, 40, 6)
    return X, y

Using TensorFlow backend.


In [17]:
X, y = generate_data('./data/sentiment_word_tagging_train.csv')

finish word_to_index


In [21]:
y

array([[[ 0.,  0.,  0.,  0.,  0.,  1.],
        [ 0.,  0.,  0.,  0.,  0.,  1.],
        [ 0.,  0.,  0.,  0.,  0.,  1.],
        ..., 
        [ 1.,  0.,  0.,  0.,  0.,  0.],
        [ 1.,  0.,  0.,  0.,  0.,  0.],
        [ 1.,  0.,  0.,  0.,  0.,  0.]],

       [[ 0.,  0.,  0.,  0.,  0.,  1.],
        [ 0.,  0.,  0.,  0.,  0.,  1.],
        [ 0.,  0.,  0.,  0.,  0.,  1.],
        ..., 
        [ 1.,  0.,  0.,  0.,  0.,  0.],
        [ 1.,  0.,  0.,  0.,  0.,  0.],
        [ 1.,  0.,  0.,  0.,  0.,  0.]],

       [[ 0.,  0.,  0.,  0.,  0.,  1.],
        [ 0.,  0.,  0.,  0.,  0.,  1.],
        [ 0.,  0.,  0.,  0.,  0.,  1.],
        ..., 
        [ 1.,  0.,  0.,  0.,  0.,  0.],
        [ 1.,  0.,  0.,  0.,  0.,  0.],
        [ 1.,  0.,  0.,  0.,  0.,  0.]],

       ..., 
       [[ 0.,  0.,  0.,  0.,  0.,  1.],
        [ 0.,  0.,  0.,  0.,  0.,  1.],
        [ 0.,  0.,  0.,  0.,  0.,  1.],
        ..., 
        [ 1.,  0.,  0.,  0.,  0.,  0.],
        [ 1.,  0.,  0.,  0.,  0.,  0.],
     

# 1. 模型训练

In [ ]:
maxlen = 40
word_size = 128
from keras.layers import Dense, Embedding, LSTM, TimeDistributed, Input, Bidirectional
from keras.models import Model

sequence = Input(shape=(maxlen,), dtype='int32')
embedded = Embedding(len(all_words)+1, word_size, input_length=maxlen, mask_zero=True)(sequence)
blstm = Bidirectional(LSTM(64, return_sequences=True), merge_mode='sum')(embedded)
output = TimeDistributed(Dense(6, activation='softmax'))(blstm)
model = Model(input=sequence, output=output)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

batch_size = 1024
history = model.fit(X, y, batch_size=batch_size, epochs=10)


# 2. 模型测试

In [3]:
from keras.models import load_model
model = load_model('./model/sentiment_model.hdf5')
model.predict()


x1

x1 = np.array([X[0]])

y1 = y[0]
predict = model.predict(x1)

predict = predict.reshape(40, 6)


predict[0:10]

res = []
for row in predict:
    res.append(np.argmax(row))
res

5728/5728 [==============================] - 3s     


[0.25671975493597587, 0.91665562338003237]

# 3. 分词

In [23]:
import pandas as pd
import numpy as np
from keras.models import load_model
data = pd.read_csv('./data/test.csv', delimiter='\t')
content = data.iloc[0]["content"]
content + '。'
model = load_model('./model/sentiment_model.hdf5')

In [3]:
# 情感词转移矩阵
zy = {'BE': 0.8388004045000632,
 'BM': 0.1611995954999368,
 'EB': 0.08903315557616508,
 'EN': 0.990513604698914,
 'ES': -0.07954676027507912,
 'ME': 0.7658009307911725,
 'MM': 0.2341990692088275,
 'NB': 0.0655631997557599,
 'NN': 0.9842186311840461,
 'NS': -0.04978183093980593,
 'SB': 0.05394649568870219,
 'SN': 0.00023000846530342705,
 'SS': 0.9458234958459945
}

In [4]:
A = zy
zy['BE'] = A['BE'] / (A['BE'] + A['BM'])
zy['BM'] = 1 - zy['BE']

zy['EB'] = A['EB'] / (A['EN'] + A['ES'] + A['EB'])
zy['EN'] = A['EN'] / (A['EN'] + A['ES'] + A['EB'])
zy['ES'] = 1 - zy['EB'] - zy['EN']

zy['ME'] = A['ME'] / (A['ME'] + A['MM'])
zy['MM'] = 1 - zy['ME']

zy['NB'] = A['NB'] / (A['NN'] + A['NS'] + A['NB'])
zy['NN'] = A['NN'] / (A['NN'] + A['NS'] + A['NB'])
zy['NS'] = 1 - zy['NB'] - zy['NN']

zy['SB'] = A['SB'] / (A['SN'] + A['SS'] + A['SB'])
zy['SN'] = A['NN'] / (A['SN'] + A['SS'] + A['SB'])
zy['SS'] = 1 - zy['SB'] - zy['SN']

In [24]:
import re

zy = {i:np.log(zy[i]) for i in zy.keys()}
def viterbi(nodes):
    """
    维特比译码：除了第一层以外，每一层有4个节点。
    计算当前层（第一层不需要计算）四个节点的最短路径：
       对于本层的每一个节点，计算出路径来自上一层的各个节点的新的路径长度（概率）。保留最大值（最短路径）。
       上一层每个节点的路径保存在 paths 中。计算本层的时候，先用paths_ 暂存，然后把本层的最大路径保存到 paths 中。
       paths 采用字典的形式保存（路径：路径长度）。
       一直计算到最后一层，得到四条路径，将长度最短（概率值最大的路径返回）
    """
    # 第一层，三个节点
    paths = {'B': nodes[0]['B'], 'S':nodes[0]['S'], 'N':nodes[0]["N"]} 
    for layer in range(1, len(nodes)):  # 后面的每一层
        paths_ = paths.copy()  # 先保存上一层的路径
        # node_now 为本层节点， node_last 为上层节点
        paths = {}  # 清空 path 
        for node_now in nodes[layer].keys():
            # 对于本层的每个节点，找出最短路径
            sub_paths = {} 
            # 上一层的每个节点到本层节点的连接
            for path_last in paths_.keys():
                if path_last[-1] + node_now in zy.keys(): # 若转移概率不为 0 
                    sub_paths[path_last + node_now] = paths_[path_last] + nodes[layer][node_now] + zy[path_last[-1] + node_now]
            # 最短路径,即概率最大的那个
            sr_subpaths = pd.Series(sub_paths)
            sr_subpaths = sr_subpaths.sort_values()  # 升序排序
            node_subpath = sr_subpaths.index[-1]  # 最短路径
            node_value = sr_subpaths[-1]   # 最短路径对应的值
            # 把 node_now 的最短路径添加到 paths 中
            paths[node_subpath] = node_value
    # 所有层求完后，找出最后一层中各个节点的路径最短的路径
    sr_paths = pd.Series(paths)
    sr_paths = sr_paths.sort_values()  # 按照升序排序
    return sr_paths.index[-1]  # 返回最短路径（概率值最大的路径）

def simple_cut(s, tokenizer):
    if s:
        tmp = sent_to_vec(s, tokenizer)[0]
        tmp.extend([0]*(40-len(s)))
        tmp = np.array(tmp)
        tmp = tmp.reshape(-1, 40)
        r = model.predict(tmp, verbose=False)[0][:len(s)]
        print(r)
        r = np.log(r)
        nodes = [dict(zip(['B','M','E','S', 'N'], i[1:])) for i in r]
        t = viterbi(nodes)
        words = []
        for i in range(len(s)):
            if t[i] in ['S', 'B', 'N']:
                words.append(s[i])
            else:
                words[-1] += s[i]
        return words
    else:
        return []

# not_cuts = re.compile(u'[。，、？！\.\?,!]')
# def cut_word(s):
#     result = []
#     j = 0
#     for i in not_cuts.finditer(s):
#         result.extend(simple_cut(s[j:i.start()]))
#         result.append(s[i.start():i.end()])
#         j = i.end()
#     result.extend(simple_cut(s[j:]))
#     return result

def sent_to_vec(s, tokenizer):
    res = word_to_index(s, "content", tokenizer)
    for item in res:
        if len(item) == 0:
            item.append(0)
    return res
token = initTokenizer()
print(simple_cut('我觉得不是正品', token))

[[  3.97639133e-06   2.00652881e-04   1.99369359e-04   7.59826333e-04
    1.44626872e-04   9.98691499e-01]
 [  3.40076014e-07   8.66002170e-04   1.12468369e-04   4.29953798e-05
    3.07019873e-05   9.98947442e-01]
 [  2.43994577e-06   1.06700500e-02   3.10417404e-03   2.01119389e-03
    3.97963775e-03   9.80232477e-01]
 [  1.87809121e-06   3.40575904e-01   1.18051685e-01   2.02678214e-03
    5.32808597e-04   5.38810909e-01]
 [  2.29584166e-05   5.37308026e-03   7.65544176e-01   1.96815282e-02
    3.98820906e-04   2.08979353e-01]
 [  1.50923215e-05   2.73587078e-01   5.62995672e-01   3.16475257e-02
    2.18529356e-04   1.31536067e-01]
 [  4.29033171e-05   1.93575269e-03   2.86455341e-02   8.73806596e-01
    2.41114351e-04   9.53281447e-02]]
['我', '觉', '得', '不', '是', '正', '品']


In [22]:
model.evaluate(X, y)

50784/50796 [============================>.] - ETA: 0s

[0.24350443599153498, 0.92025650792332536]